#Set Environment

In [ ]:
%%bash
pip install itables
pip install matplotlib
pip install pandas
pip install scipy
pip install numpy
pip install seaborn

#Import libraries

In [6]:
import os as os
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import shapiro
import seaborn as sns
import matplotlib.pyplot as plt
import shutil
import glob as glob
import itables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

#Define functions

In [ ]:
# Loading data from CSV files
genome_enc = pd.read_csv("~/Downloads/genome_enc.csv")
genes = pd.read_csv("~/Downloads/Gene_tsv.tsv", sep=",")
genes.drop(genes.columns[0], axis=1, inplace=True)
hmm = pd.read_csv("~/Downloads/RiboV1.4/RiboV1.4_HMMatches.tsv", sep="\t")

In [ ]:
# Classify genes based on 'hmm' and 'genes' dataframes
hmm_with_struct = classify_struct_of_genes(hmm)
complete_genes = remove_lines_of_uncompleted_genes(genes)

complete_genes_sorted = complete_genes.sort_values(by=['seqid', 'ENC'])
complete_genes_sorted['ENC_rank'] = complete_genes_sorted.groupby('seqid')['ENC'].rank().astype(int)

hmm_with_struct_slim = hmm_with_struct[['ORFID', 'struct', 'Family', 'New_Name']].drop_duplicates()
hmm_with_struct_slim = hmm_with_struct_slim.drop_duplicates(subset='ORFID', keep=False, inplace=False)

hmm_genes = pd.merge(complete_genes_sorted, hmm_with_struct_slim, on='ORFID', how='inner')
hmm_genes['prop_ENC'] = hmm_genes['ENC'] / hmm_genes.groupby('seqid')['ENC'].transform('max')

# Zoom-in on analayzed genes
hmm_genes = hmm_genes[~(hmm_genes['Family'].str.contains(r'\.') | hmm_genes['Family'].isna())]
Families_interest = ['Tombusviridae', 'Steitzviridae', 'Botourmiaviridae',
                     'Fiersviridae', 'Mymonaviridae', 'Virgaviridae', 'Astroviridae', 'Cystoviridae']
hmm_genes = hmm_genes[hmm_genes['Family'].isin(Families_interest)]

grouped_hmm_genes = hmm_genes.groupby(['Family', 'New_Name', 'struct']).size().reset_index(name='count')

summary_table = hmm_genes.groupby(['Family', 'New_Name', 'struct']).agg({
    'ENC_rank': 'median',  # Calculate the median of genes for each gene type
    'ENC': 'mean',  # Calculate the average ENC value for each gene type
    'prop_ENC': 'mean'  # Calculate the average prop_ENC value for each gene type
}).reset_index()

summary_hmm_genes_table = pd.merge(grouped_hmm_genes, summary_table, on=['Family', 'New_Name', 'struct'])

new_column_names = {'New_Name': 'Function',
                    'struct': 'Category'}
summary_hmm_genes_table.rename(columns=new_column_names, inplace=True)


In [ ]:
complete_genes_sorted = complete_genes.sort_values(by=['seqid', 'ENC'])
complete_genes_sorted['ENC_rank'] = complete_genes_sorted.groupby('seqid')['ENC'].rank().astype(int)

hmm_with_struct_slim = hmm_with_struct[['ORFID', 'struct', 'Family', 'New_Name']].drop_duplicates()
hmm_with_struct_slim = hmm_with_struct_slim.drop_duplicates(subset='ORFID', keep=False, inplace=False)

hmm_genes = pd.merge(complete_genes_sorted, hmm_with_struct_slim, on='ORFID', how='inner')
hmm_genes['prop_ENC'] = hmm_genes['ENC'] / hmm_genes.groupby('seqid')['ENC'].transform('max')

In [4]:
def count_number_of_unique_struct_columns_by_value(df):
    """
    Count the occurrences of each unique value in the 'struct' column of the DataFrame.

    Parameters:
        df (DataFrame): The input DataFrame.

    Returns:
        Series: A Series containing the count of occurrences for each unique value in the 'struct' column.
    """
    return df.groupby('struct')['struct'].count()

def classify_struct_of_genes(df):
    """
    Classify the 'struct' column of the DataFrame based on the 'New_Name' column.

    Parameters:
        df (DataFrame): The input DataFrame.

    Returns:
        DataFrame: A copy of the input DataFrame with a new 'struct' column containing the classifications.
    """
    df2 = df.copy()
    list_of_struct = ["CP", "Coat", "Glycoprotein", "Nucleoprotein", "Membrane", "Capsid", "Envelope"]
    list_of_non_struct = ["RdRp", "Helicase", "Polymerase", "Methyltransferase", "Cap_MTase-GTase", "MTase", "GTase", "Pro", "Maturation"]
    df2['struct'] = np.where(df2['New_Name'].str.lower().str.contains('|'.join(map(str.lower, list_of_struct))), 'Structural',
                            np.where(df2['New_Name'].str.lower().str.contains('|'.join(map(str.lower, list_of_non_struct))), 'Non-Structural', 'Unknown'))
    return df2

def intersect_genes_by_ORFID(df1, df2):
    """
    Get the intersection of two DataFrames based on the 'ORFID' column.

    Parameters:
        df1 (DataFrame): First input DataFrame.
        df2 (DataFrame): Second input DataFrame.

    Returns:
        DataFrame: A new DataFrame containing the intersection of the two DataFrames.
    """
    return df1[df1['ORFID'].isin(df2['ORFID'])]

def remove_row_if_ORFID_not_in_genes(hmm_with_struct, genes):
    """
    Remove rows from 'hmm_with_struct' DataFrame if 'ORFID' is not present in the 'genes' DataFrame.

    Parameters:
        hmm_with_struct (DataFrame): DataFrame with genes classified based on structure.
        genes (DataFrame): The input DataFrame containing genes.

    Returns:
        DataFrame: The 'hmm_with_struct' DataFrame with rows removed for unmatched ORFIDs.
    """
    return hmm_with_struct[hmm_with_struct['ORFID'].isin(genes['ORFID'])]

def remove_lines_of_uncompleted_genes(df):
    """
    Remove rows from DataFrame where the 'partial' column contains specific values.

    Parameters:
        df (DataFrame): The input DataFrame.

    Returns:
        DataFrame: A new DataFrame with rows removed based on the 'partial' column conditions.
    """
    return df[~df['partial'].astype(str).isin(['01', '10', '1', '11'])]

#### statitical testing ####

# returns list of lists, every list stores values of a feature for a specic taxon
def create_list_of_list_of_var(stacked_df, variable):
    lst_of_samples = []
    tmp = stacked_df.groupby('Family')[variable].apply(list).reset_index(name=variable)
    for index, row in tmp.iterrows():
        lst_of_samples.append(tmp.iat[index, 1])
    return lst_of_samples


def statistical_analysis(stacked_df, variable, Negative=False):
    print('\nVariable = ', variable)
    normally_distributed = all_samples_are_noramlly_distributed(stacked_df, variable)
    if (normally_distributed != True):
        print("not all samples are normally distributed")

    samples_lst = create_list_of_list_of_var(stacked_df, variable)
    equal_var_test_res = test_variances(*samples_lst, normally_distributed=normally_distributed)

    if (normally_distributed != True):
        test_means_non_normally_distributed(*samples_lst)
    else:
        test_means_normally_distributed(*samples_lst, df=stacked_df, variable=variable,
                                        variances_are_equal=equal_var_test_res)

def all_samples_are_noramlly_distributed(stacked_df, variable):
    lst_of_samples = create_list_of_list_of_var(stacked_df, variable)
    for i in range(len(lst_of_samples)):
        if test_normality(lst_of_samples[i]) == False:
            return False
    return True

def facet_hist(stacked_df, variable, max_tik=65, step_size=1, start_tik=25,
               group_var="Family", Row_var="Family", wp="placehodler"):
    # Initialize the FacetGrid object
    stacked_df.loc[stacked_df.index[stacked_df[variable].isna()], variable] = 0

    stacked_df = stacked_df.sort_values([group_var, Row_var])
    pal = sns.cubehelix_palette(len(stacked_df[group_var].unique()), rot=-.25, light=.7)
    g = sns.FacetGrid(stacked_df, row=Row_var, hue=group_var, aspect=15, height=1.2, palette='viridis')

    # Draw the densities in a few steps
    g.map(sns.kdeplot, variable,
          bw_adjust=.5, clip_on=False,
          fill=True, alpha=1, linewidth=.5)
    g.map(sns.kdeplot, variable, clip_on=False, color="w", lw=.2, bw_adjust=.5)

    # passing color=None to refline() uses the hue mapping
    g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)

    # Define and use a simple function to label the plot in axes coordinates
    def label(variable, color, label=Row_var):
        ax = plt.gca()
        ax.text(0.005, .3, label, fontweight="bold", fontsize=14,
                ha="left", va="center", transform=ax.transAxes)

    # Set the subplots to overlap
    g.figure.subplots_adjust(hspace=+.25)
    # Remove axes details that don't play well with overlap
    g.set_titles("")
    g.set(xticks=list(range(start_tik, max_tik, step_size)), xlabel=variable)
    g.set_xlabels(label=None, clear_inner=True, fontsize=10, va="top")

    # for ax, label in zip(g.axes.flat, g.row_names):
    for ax, row_label in zip(g.axes.flat, g.row_names):
        ax.annotate(row_label, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                    xycoords=ax.yaxis.label, textcoords='offset points',
                    ha='right', fontsize=10, va='center', fontstyle='italic')
    # Add a legend
    g.add_legend(fontsize=10)
    g.tick_params(axis='x', which='major', labelsize=15)

    plt.title(wp + ' ' + Row_var + '  distribution by ' + group_var, fontsize=20, loc='center', pad=(65 * (len(stacked_df[group_var].unique()))))
    plt.savefig(str(wp + '_' + group_var + '_' + variable + '.pdf'), dpi=300, bbox_inches='tight')

# test given samples if normally distributed using the Shapiro–Wilk test
# H0: samples are normally distributed

def test_normality(df, variable_name):
    """
    Perform the Shapiro-Wilk test to test normality of a variable in the DataFrame.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing the variable to test.
        variable_name (str): The name of the variable to test for normality.
        
    Returns:
        (bool, float): A tuple containing a boolean indicating normality and the test p-value.
    """
    if variable_name not in df.columns:
        raise ValueError(f"Variable '{variable_name}' not found in the DataFrame.")
    
    variable_data = df[variable_name].dropna()
    stat, p_value = shapiro(variable_data)
    
    alpha = 0.05  # significance level
    is_normal = p_value > alpha
    
    return is_normal, p_value

def kruskal_test(group):
    for enc_column in ["ENC", "ENC_rank", "prop_ENC"]:
        data_columns = [group[group["Family"] == group.iloc[0]["Family"]][enc_column] for _, group in group.groupby(["Category", "Function"])]
        data_columns = [data for data in data_columns if len(data) > 1]  # Removing groups with insufficient data
        if len(data_columns) > 1:
            stat, p_value = kruskal(*data_columns)
            print(f"Family: {group.iloc[0]['Family']}, Column: {enc_column}")
            print(f"Kruskal-Wallis test statistic: {stat}, p-value: {p_value}")
            if p_value < 0.05:
                print("The distributions are statistically different.")
            else:
                print("The distributions are not statistically different.")
        else:
            print(f"Family: {group.iloc[0]['Family']}, Column: {enc_column}")
            print("Insufficient variability: Kruskal-Wallis test cannot be performed.")

def plot_boxplot(df, value_col, category_col, family_name):
    """
    Create a box plot for values in the DataFrame separated by a given category column.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        value_col (str): The column name for the values to be plotted on the y-axis.
        category_col (str): The column name for the categories to be plotted on the x-axis.
        family_name (str): The family name to be used in the plot title and the output filename.

    Returns:
        None
    """
    plt.figure(figsize=(8, 6))  # Set the size of the plot (optional)

    sns.boxplot(x=category_col, y=value_col, data=df)

    # Add labels and title
    plt.xlabel('Category')
    plt.ylabel('Value')
    plt.title(f'Box Plot of {value_col} by Category - Family: {family_name}')

    # Create the output filename
    output_file = f'{family_name}_{value_col}_boxplot.pdf'

    # Save the plot as a PDF file
    plt.savefig(output_file, format='pdf')

    plt.show()

In [ ]:
    ###hist plots###
    for family in hmm_genes['Family'].unique():
        facet_hist(hmm_genes[hmm_genes['Family'] == family], 'ENC', max_tik=65,
                   step_size=1, start_tik=20, group_var="Category", Row_var='Category', wp=family)

    for family in hmm_genes['Family'].unique():
        facet_hist(hmm_genes[hmm_genes['Family'] == family], 'ENC', max_tik=65,
                   step_size=1, start_tik=20, group_var="Function", Row_var='Function', wp=family)

    hmm_genes['prop_ENC'] = hmm_genes['prop_ENC'] * 100


    for family in hmm_genes['Family'].unique():
        facet_hist(hmm_genes[hmm_genes['Family'] == family], 'prop_ENC', max_tik=100,
                   step_size=5, start_tik=0, group_var="Category", Row_var='Category', wp=family)


    for family in hmm_genes['Family'].unique():
        facet_hist(hmm_genes[hmm_genes['Family'] == family], 'ENC_rank', max_tik=10,
                   step_size=1, start_tik=0, group_var="Category", Row_var='Category', wp=family)
        
    ###statistical analysis###

    grouped = hmm_genes.groupby('Family')

    grouped.apply(kruskal_test)

    for family, group in grouped:
        test_normality(group, 'ENC')
        test_normality(group, 'prop_ENC')
        test_normality(group, 'ENC_rank')
        plot_boxplot(group, 'ENC', 'Category', family)
        plot_boxplot(group, 'prop_ENC', 'Category', family)
        plot_boxplot(group, 'ENC_rank', 'Category', family)